# Single-GPU Training with Amazon SageMaker on Amazon SageMaker Studio Lab

In [ ]:
!pip install sagemaker --upgrade -q
!pip install ipywidgets -q
!pip install tensorflow -q

**Step 1:** Import essentials packages, start a sagemaker session and specify the bucket name you created in the pre-requsites section of this workshop.

In [ ]:
import os
import boto3
import time
import numpy as np
import sagemaker

If using temporary credentials, copy the output of `aws sts get-session-token --duration-seconds <DURATION>` and assign it to the `credentials` variable below.

If you're using long-term credentials, ignore this cell

In [ ]:
credentials = {
    "Credentials": {
        "AccessKeyId": "<AWS_ACCESS_KEY_ID>",
        "SecretAccessKey": "<AWS_SECRET_ACCESS_KEY>",
        "SessionToken": "<AWS_SESSION_TOKEN>",
    }
}
os.environ["AWS_ACCESS_KEY_ID"] = credentials['Credentials']['AccessKeyId']
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials['Credentials']['SecretAccessKey']
os.environ["AWS_SESSION_TOKEN"] = credentials['Credentials']['SessionToken']

Assign the sagemaker studio lab ARN

In [ ]:
sagemaker_role='arn:aws:iam::<ACCOUNT_ID>:role/SageMakerStudioLabRole'

In [ ]:
sess = boto3.Session()
sm   = sess.client('sagemaker')
sagemaker_session = sagemaker.Session()
role = sagemaker_role

bucket_name    = sagemaker_session.default_bucket()
jobs_folder    = 'jobs'
dataset_folder = 'datasets'

![](https://miro.medium.com/max/1000/0*GRfvsrvtfpRm400-)

In [ ]:
!python generate_cifar10_tfrecords.py --data-dir=cifar10-dataset
datasets = sagemaker_session.upload_data(path='cifar10-dataset', key_prefix=f'{dataset_folder}/cifar10-tfrecords')
datasets

**Step 2:** Specify hyperparameters, instance type and number of instances to distribute training to. 

In [ ]:
job_name   = f'tf-single-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S-%j", time.gmtime())}'
output_path = f's3://{bucket_name}/{jobs_folder}'

metric_definitions = [{'Name': 'Validation Accuracy', 'Regex': 'Validation Accuracy: ([0-9\\.]+)'}]

hyperparameters = {'epochs'       : 50, 
                   'learning-rate': 0.01,
                   'momentum'     : 0.95,
                   'weight-decay' : 2e-4,
                   'optimizer'    : 'adam',
                   'batch-size'   : 256,
                   'model-type'   : 'custom'}


In [ ]:
from sagemaker.debugger import TensorBoardOutputConfig

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path=f's3://{bucket_name}/tensorboard_logs')

In [ ]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(entry_point         = 'cifar10-tf2.py', 
                           source_dir           = 'code',
                           output_path          = output_path + '/',
                           code_location        = output_path,
                           role                 = role,
                           instance_count       = 1,
                           instance_type        = 'ml.p3.2xlarge',
                           framework_version    = '2.4', 
                           py_version           = 'py37',
                           metric_definitions   = metric_definitions,
                           hyperparameters      = hyperparameters,
                           tensorboard_output_config = tensorboard_output_config)

**Step 4:** Specify dataset locations in Amazon S3 and then call the fit function.

In [ ]:
estimator.fit({'train': datasets,
                    'validation': datasets,
                    'eval': datasets}, 
                  job_name=job_name, wait=True)

## Clean up

Delete S3 artifacts

In [ ]:
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(bucket_name)
s3_bucket.objects.filter(Prefix=f"{jobs_folder}/").delete()
s3_bucket.objects.filter(Prefix=f"{dataset_folder}/").delete()

print(f"\nDeleted contents of {bucket}/{jobs_folder}")
print(f"\nDeleted contents of {bucket}/{dataset_folder}")